* Neural Network by Chainer

In [4]:
import time
import numpy as np
import pandas as pd
import chainer
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from sklearn import datasets

In [5]:
# 教師データ

N = 100
in_size = 4
out_size = 3

iris = datasets.load_iris()
data = pd.DataFrame(data= np.c_[iris["data"], iris["target"]], columns= iris["feature_names"] + ["target"])
data = np.array(data.values)

perm = np.random.permutation(len(data))
data = data[perm]
train, test = np.split(data, [N])

train_x, train_y, test_x, test_y = [], [], [], []
for t in train:
    train_x.append(t[0:4])
    train_y.append(t[4])
for t in test:
    test_x.append(t[0:4])
    test_y.append(t[4])
train_x = np.array(train_x, dtype="float32")
train_y = np.array(train_y, dtype="int32")
test_x = np.array(test_x, dtype="float32")
test_y = np.array(test_y, dtype="int32")

In [8]:
class NN(chainer.Chain):
    
    def __init__(self, in_size, hidden_size, out_size):
        
        super(NN, self).__init__(
            xh = L.Linear(in_size, hidden_size),
            hh = L.Linear(hidden_size, hidden_size),
            hy = L.Linear(hidden_size, out_size)
        )
 
    def __call__(self, x, y=None, train=False):
        
        x = chainer.Variable(x)
        if train:
            y = chainer.Variable(y)
            
        h = F.sigmoid(self.xh(x))
        h = F.sigmoid(self.hh(h))
        y_ = F.softmax(self.hy(h))
        
        if train:
            loss, accuracy = F.softmax_cross_entropy(y_, y), F.accuracy(y_, y)
            return loss, accuracy
        else:
            return np.argmax(y_.data)
 
    def reset(self):
        
        self.zerograds()

In [9]:
HIDDEN_SIZE = 20

model = NN(in_size=in_size, hidden_size=HIDDEN_SIZE, out_size=out_size)
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

In [10]:
EPOCH_NUM = 100
BATCH_SIZE = 20

st = time.time()
for epoch in range(EPOCH_NUM):
    
    # ミニバッチ学習
    perm = np.random.permutation(N) # ランダムな整数列リストを取得
    total_loss = 0
    total_accuracy = 0
    for i in range(0, N, BATCH_SIZE): 
        
        x = train_x[perm[i:i+BATCH_SIZE]]
        y = train_y[perm[i:i+BATCH_SIZE]]
        
        model.reset()
        loss, accuracy = model(x=x, y=y, train=True)
        loss.backward()
        loss.unchain_backward()
        total_loss += loss.data
        total_accuracy += accuracy.data
        optimizer.update()
        
    if (epoch+1) % 10 == 0:
        ed = time.time()
        print("epoch:\t{}\ttotal loss:\t{}\tmean accuracy:\t{}\ttime:\t{}".format(epoch+1, total_loss, total_accuracy/(N/BATCH_SIZE), ed-st))
        st = time.time()

epoch:	10	total loss:	5.421875953674316	mean accuracy:	0.33999999761581423	time:	0.14729762077331543
epoch:	20	total loss:	5.325374126434326	mean accuracy:	0.6600000023841858	time:	0.14753317832946777
epoch:	30	total loss:	5.1339027881622314	mean accuracy:	0.6600000023841858	time:	0.14910244941711426
epoch:	40	total loss:	4.833737730979919	mean accuracy:	0.6600000143051148	time:	0.1455545425415039
epoch:	50	total loss:	4.529348075389862	mean accuracy:	0.6600000023841858	time:	0.1494755744934082
epoch:	60	total loss:	4.319056153297424	mean accuracy:	0.6600000023841858	time:	0.14522838592529297
epoch:	70	total loss:	4.171298801898956	mean accuracy:	0.7400000095367432	time:	0.15001463890075684
epoch:	80	total loss:	4.040274083614349	mean accuracy:	0.8899999856948853	time:	0.1461775302886963
epoch:	90	total loss:	3.9164429306983948	mean accuracy:	0.95	time:	0.14641165733337402
epoch:	100	total loss:	3.791893422603607	mean accuracy:	0.9599999904632568	time:	0.14478468894958496


In [11]:
res = []
for x, y in zip(test_x, test_y):
    y_ = model(x=x.reshape(1,len(x)), train=False)
    if y == y_:
        res.append(1)
    else:
        res.append(0)
accuracy = sum(res)/len(res)
print("test data accuracy: ", accuracy)

test data accuracy:  0.98


In [12]:
!python --version

Python 3.6.3


In [13]:
!pip freeze

absl-py==0.2.0
astor==0.6.2
backcall==0.1.0
bleach==1.5.0
boto==2.49.0
boto3==1.7.73
botocore==1.10.73
bz2file==0.98
certifi==2018.4.16
chainer==4.0.0
chainercv==0.10.0
chardet==3.0.4
cntk-gpu==2.5.1
cupy==4.0.0
cycler==0.10.0
Cython==0.28.2
decorator==4.3.0
dm-sonnet==1.20
docutils==0.14
edward==1.3.5
entrypoints==0.2.3
fastrlock==0.3
filelock==3.0.4
future==0.16.0
gast==0.2.0
gensim==3.5.0
graphviz==0.8.3
grpcio==1.11.0
h5py==2.7.1
html5lib==0.9999999
idna==2.6
ipykernel==4.8.2
ipython==6.3.1
ipython-genutils==0.2.0
ipywidgets==7.2.1
jedi==0.12.0
Jinja2==2.10
jmespath==0.9.3
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.3
jupyter-console==5.2.0
jupyter-core==4.4.0
Keras==2.1.6
kiwisolver==1.0.1
Lasagne==0.2.dev1
leveldb==0.194
Mako==1.0.7
Markdown==2.6.11
MarkupSafe==1.0
matplotlib==2.2.2
mecab-python3==0.7
mistune==0.8.3
mxnet-cu90==1.1.0.post0
nbconvert==5.3.1
nbformat==4.4.0
networkx==2.1
nose==1.3.7
notebook==5.4.1
numpy==1.14.3
pandas==0.22.0
pandocfilters==1.4.2
parso==0